# Association analysis

This notebook focuses on creating and analysing assocation rules found in posts.

We first load all the libraries we may use throughout the project

For a detailed explanation of association rules based on the apriori algorithm, we found this website to be particularly helpful:
https://towardsdatascience.com/association-rules-2-aa9a77241654

In [1]:
#Import graphing utilities
%matplotlib inline
import matplotlib.pyplot as plt
import operator

# Import useful mathematical libraries
import numpy as np
import pandas as pd

# Import useful Machine learning libraries
import gensim
from sklearn.cluster import KMeans

# Import utility files
from utils import save_object, load_object, make_post_clusters, make_clustering_objects, get_support_2_cluster, get_support_1_cluster, get_rule_pairs

from orangecontrib.associate.fpgrowth import *

### Set model name

Before begining the rest of this project, we select a name for our model. This name will be used to save and load the files for this model

In [2]:
# Set the model we are going to be analyzing
model_name = "DrugAbuse"
subreddit_names = ['Marijuana','Cigarettes','Alcohol','Opiates','Cocaine']

### Prepare data

We now load and process the data we will need for the rest of this project

In [3]:
dfs = []
scores = []
num_comments = []
index = 0
for sub in subreddit_names:
    dfs.append(load_object('objects/', model_name + "-" + sub + '_df'))
    scores.append(list(dfs[index]['score']))
    num_comments.append(list(dfs[index]['num_comments']))
    index = index + 1
# Load Our Saved matricies
PostsByWords = []
totalPosts = 0
for i in range(len(subreddit_names)):
    PostsByWords.append(load_object('matricies/', model_name + "-" + subreddit_names[i] + "PostsByWords"))
    totalPosts += PostsByWords[i].shape[0]
WordsByFeatures = load_object('matricies/', model_name + "-WordsByFeatures")

# Generate the posts by Features matrix through matrix multiplication
PostsByFeatures = []
for word in PostsByWords:
    PostsByFeaturesM = word.dot(WordsByFeatures)
    PostsByFeatures.append(np.matrix(PostsByFeaturesM))

model = gensim.models.Word2Vec.load('models/' + model_name + '.model')

vocab_list = sorted(list(model.wv.vocab))

# Initialize a word clustering to use
num_word_clusters = 100
kmeans =  load_object('clusters/', model_name + '-words-cluster_model-' + str(num_word_clusters))

clusters = make_clustering_objects(model, kmeans, vocab_list, WordsByFeatures)

clusterWords = list(map(lambda x: list(map(lambda y: y[0] , x["word_list"])), clusters))

from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer(vocabulary = vocab_list, analyzer = (lambda lst:list(map((lambda s: s), lst))), min_df = 0)

# Make Clusters By Words Matrix
ClustersByWords = countvec.fit_transform(clusterWords)

# Ensure consistency
len(WordsByFeatures) == ClustersByWords.shape[1]

# take the transpose of Clusters
WordsByCluster = ClustersByWords.transpose()

# Multiply Posts by Words by Words By cluster to get Posts By cluster
PostsByClusters = []
for word in PostsByWords:
    PostsByClusters.append(word.dot(WordsByCluster))

/Users/Alex/anaconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [4]:
PostsByWords[3].shape[0]

21537

### Create Association rules

Now that our data has been prepared, we create our itemsets, and then analyze them by creating association rules.

In [5]:
#Set Thresholds
s_threshold = 0.08
c_threshold = 0.5

In [ ]:
#This will take a long time! Once you've run it once it should save in the next code block so just load it after that.
itemsets = []

for post in PostsByClusters:
    itemsets.append(dict(frequent_itemsets(post > 0, s_threshold)))

In [ ]:
#Save itemset so you can load it in later
save_object(itemsets,'objects/',model_name+'-itemset'+str(s_threshold)+'Threshold')
itemsets = load_object('objects/',model_name+'-itemset'+str(s_threshold)+'Threshold')

In [ ]:
#Delete any rules that aren't one to one.
for itemset in itemsets:
    for key in itemset.copy().keys():
        if len(key) > 2:
            del itemset[key]
print(len(itemsets[2]))


In [6]:
#Save update itemset.
#save_object(itemsets,'objects/',model_name+'-itemset-'+str(s_threshold)+'ThresholdUpdated')
itemsets = load_object('objects/',model_name+'-itemset-'+str(s_threshold)+'ThresholdUpdated')

In [7]:
#Created Association Rules and Slim them down
rules_slim = []
for i in range(len(itemsets)):
    try:
        rules = [(P, Q, supp, conf, conf/(itemsets[i][Q]/PostsByClusters[i].shape[0]))
             for P, Q, supp, conf in association_rules(itemsets[i],c_threshold)]
    except Exception as e:
        print('Error Here: ' + str(i))
        print(e)
        continue
    rules_sorted = sorted(rules, key=lambda x: -x[4])
    #Adjust how many items can be in a rule
    rules_slimmed = []
    for rule in rules_sorted:
        #Filter by rule ratio (1:1,2:1,etc) and confidence.
        if (len(rule[0]) == 1) and (len(rule[1]) == 1):
            rules_slimmed.append(rule)
    rules_slim.append(rules_slimmed)

In [8]:
#This stores the order of the rule (A-->B vs B-->A) for future reference so no backwards rules are generated.
rule_order_lhs = {}
for rules in rules_slim:
    for rule in rules:
        rule_order_lhs.update({(rule[0] | rule[1]):rule[0]})
print(rule_order_lhs[frozenset({3, 47})])

frozenset({3})


In [9]:
#Get Rule pairs to pass to appending itemset dictionary function.
all_rules = get_rule_pairs(rules_slim)
print(len(all_rules[1]))


229


In [10]:
#Get a list of all rule pairs without repeats.
res = list(set(all_rules[0])^set(all_rules[1])^set(all_rules[2])^set(all_rules[3])^set(all_rules[4]))
print(len(res))
print(res[0])
arr = np.asarray(res[0])

1005
(16, 9)


In [11]:
#Pass rule pairs to appending itemset dictionary function. This will take a few hours to run.
i = 0
for pair in res:
    itemsets = get_support_2_cluster(np.asarray(pair),PostsByClusters,itemsets)
    itemsets = get_support_1_cluster([np.asarray(pair)[0]],PostsByClusters,itemsets)
    itemsets = get_support_1_cluster([np.asarray(pair)[1]],PostsByClusters,itemsets)
    if (i % 5 == 0):
        print(str(i/len(res)) + '% Done')
    i = i + 1

0.0% Done
0.004975124378109453% Done


KeyboardInterrupt: 

In [27]:
#Save itemsets. This is important, the itemsets takes a long time to run.
save_object(itemsets,'objects/',model_name+'-'+'itemset-'+str(s_threshold)+'S-'+str(c_threshold)+'CThresholdFinal')
save_object(res,'objects/','res'+model_name)
itemsets = load_object('objects/',model_name+'-'+'itemset-'+str(s_threshold)+'S-'+str(c_threshold)+'CThresholdFinal')
res = load_object('objects/','res'+model_name)
itemsetsNew = itemsets

In [28]:
#Rerun rules this time with all the itemsets needed to get the same rules across all five subreddits. 
#This needs to be run at a very low confidence or the rules will not all show up, since some will have inevitably very low confidence. 
#This shouldn't take much time at all though.
rules_slim = []
errors = 0
for i in range(len(itemsetsNew)):
    percent_done = 0
    for pair in res:
        try:
            rules = [(P, Q, supp, conf, conf/(itemsetsNew[i][Q]/PostsByClusters[i].shape[0]))
                         for P, Q, supp, conf in association_rules(itemsetsNew[i],0.01,frozenset({np.asarray(pair)[0],np.asarray(pair)[1]}))]
        except Exception as e:
            print('Error Here: ' + str(i))
            errors = errors + 1
            continue
        rules_sorted = sorted(rules, key=lambda x: -x[4])
        #Adjust how many items can be in a rule
        rules_slimmed = []
        for rule in rules_sorted:
            #Filter by rule ratio (1:1,2:1,etc) and confidence. Probably no longer necessary, this is done earlier in this code.
            #Could be used to filter confidence with a small adjustment though.
            if (len(rule[0]) == 1) and (len(rule[1]) == 1):
                rules_slimmed.append(rule)
        rules_slim.append(rules_slimmed)

In [29]:
#Delete Rules that are the same but backwards. This references the dictionary we made previously of the original rules and their order.
xtraRules = []
for rule in rules_slim:
    if (len(rule) > 1):
        key = rule[0][0] | rule[0][1]
        if (rule_order_lhs[key] == rule[0][0]):
            xtraRules.append(rule[1])
            del rule[1]
        else:
            xtraRules.append(rule[0])
            del rule[0]

In [30]:
#Create a nice print format.
fifth = len(rules_slim)/5
index = 0
i2 = 0
ruleCount = 0
printFormat = []
for rule in rules_slim:
    if index % (fifth) == 0:
        printFormat.append(["Subreddit: " + subreddit_names[i2], '','','',])
        printFormat.append(['Rule', 'Support (%)','Confidence','Lift',])
    for lhs, rhs, support, confidence,lift in rule:
        printFormat.append([", ".join([str(i+1) for i in lhs]) + "-->"+ ", ".join([str(i+1) for i in rhs]),
              str(round(support,3)/PostsByClusters[i2].shape[0] * 100),str(round(confidence,3)),str(round(lift,3))])
        ruleCount = ruleCount + 1
    index = index + 1
    if index % (fifth) == 0:
        i2 = i2 + 1
print(len(printFormat))
print(printFormat[2])

5035
['17-->10', '13.466590352074437', '0.997', '1.042']


In [31]:
#Ordering printFormat to get nice csv file with data all next to each other.
printOrderedColumns = []
for i in range(int(fifth)+2):
    row = []
    for i2 in range(len(printFormat)):
        if (i2 % (int(fifth)+2) == i):
            row.append(printFormat[i2])
            row.append(' ')
    printOrderedColumns.append(row)
printOrderedColumns[2]

#If done correctly this should output a list containing n number of rules that are 
#the same clusters for n subreddits analyzed.
print(printOrderedColumns[8])


[['90-->46', '10.295132009568677', '0.512', '1.754'], ' ', ['90-->46', '4.589362901710205', '0.339', '1.918'], ' ', ['90-->46', '6.783870480455664', '0.367', '1.734'], ' ', ['90-->46', '8.965965547662162', '0.35', '1.706'], ' ', ['90-->46', '6.579849211788896', '0.402', '1.973'], ' ']


In [32]:
#Print out the highest lift differentials for analysis of specific rules. 
#May not be as needed with the csv files of max lift and confidence sorted below, but makes things a little quicker.
index == 0
lift_differential = []
for i in range(2,len(printOrderedColumns)):
    lifts = []
    for i2 in range(len(printOrderedColumns[i])):
        if i2 % 2 == 1:
            continue
        lifts.append(float(printOrderedColumns[i][i2][3]))
    NPLifts = np.asarray(lifts)
    maxName = np.argmax(NPLifts)
    minName = np.argmin(NPLifts)
    #Editing this if statement can allow you to show only rules with a range above a threshold or min lift above a threshold.
    #Helpful for automated identification of good motif association rules.
    if max(lifts) > 2:
        lift_differential.append([round(max(lifts) - min(lifts),3),"Max: " + subreddit_names[maxName],"Min: " + subreddit_names[minName],i])
diffSort = []
for diffs in lift_differential:
    diffSort.append((printOrderedColumns[diffs[3]][0][0],diffs[0],diffs[1],diffs[2]))
diffSort = sorted(diffSort, key=lambda x: -x[1])
print(diffSort)
save_object(diffSort,'objects/',model_name+'LiftDifferentialSort')

[('40-->54', 2.971, 'Max: Cigarettes', 'Min: Opiates'), ('97-->65', 2.473, 'Max: Alcohol', 'Min: Opiates'), ('4-->48', 2.381, 'Max: Cigarettes', 'Min: Opiates'), ('63-->81', 2.298, 'Max: Cocaine', 'Min: Alcohol'), ('91-->97', 2.265, 'Max: Alcohol', 'Min: Opiates'), ('47-->48', 2.232, 'Max: Cigarettes', 'Min: Opiates'), ('48-->97', 1.851, 'Max: Alcohol', 'Min: Opiates'), ('63-->60', 1.763, 'Max: Cocaine', 'Min: Alcohol'), ('9-->48', 1.734, 'Max: Cigarettes', 'Min: Opiates'), ('9-->97', 1.665, 'Max: Alcohol', 'Min: Opiates'), ('97-->34', 1.655, 'Max: Alcohol', 'Min: Opiates'), ('47-->97', 1.588, 'Max: Alcohol', 'Min: Opiates'), ('45-->97', 1.564, 'Max: Cocaine', 'Min: Opiates'), ('88-->60', 1.485, 'Max: Cocaine', 'Min: Alcohol'), ('26-->48', 1.478, 'Max: Cigarettes', 'Min: Opiates'), ('31-->3', 1.425, 'Max: Cocaine', 'Min: Alcohol'), ('50-->97', 1.349, 'Max: Alcohol', 'Min: Opiates'), ('4-->65', 1.344, 'Max: Cigarettes', 'Min: Opiates'), ('63-->77', 1.324, 'Max: Cocaine', 'Min: Alcohol')

# Saving CSV Files for Association Analysis
CSV Files are saved to a folder titled "Association-Analysis

## Max Total Lift Sort
Sort by overall max lift

In [33]:
#Sorting rules by max total lift across all subreddits.
printOCravel = []
for i in range(len(printOrderedColumns)):
    printOCravel.append([y for x in printOrderedColumns[i] for y in x])
printOCravelCopy = printOCravel
del printOCravelCopy[0]
del printOCravelCopy[0]

#Sorting rules by max total lift across all subreddits.
#Summing Lifts
for row in printOCravelCopy:
    row.append(float(row[3]) + float(row[8]) + float(row[13]) + float(row[18]) + float(row[23]))
    
#Add Labels
labels = []
for i in range(2):
    labels.append([y for x in printOrderedColumns[i] for y in x])
labels[1].append('Lifts Sum')
labels[0].append('')
labels[0].append('')
print(labels)

#Sort by the sum calculated
printOCravelCopy[4]
liftSortedPrint = sorted(printOCravelCopy, key=lambda x: -x[25])
liftSortedPrint = labels + liftSortedPrint

[['Subreddit: Marijuana', '', '', '', ' ', 'Subreddit: Cigarettes', '', '', '', ' ', 'Subreddit: Alcohol', '', '', '', ' ', 'Subreddit: Opiates', '', '', '', ' ', 'Subreddit: Cocaine', '', '', '', ' ', '', ''], ['Rule', 'Support (%)', 'Confidence', 'Lift', ' ', 'Rule', 'Support (%)', 'Confidence', 'Lift', ' ', 'Rule', 'Support (%)', 'Confidence', 'Lift', ' ', 'Rule', 'Support (%)', 'Confidence', 'Lift', ' ', 'Rule', 'Support (%)', 'Confidence', 'Lift', ' ', 'Lifts Sum']]


In [35]:
#Write to a CSV File
import csv
import pandas as pd
with open('Association-Analysis/'+model_name+'Associations'+str(s_threshold)+'S'+str(c_threshold)+'C_Threshold_MaxLiftSort.csv','w') as csvfile:
    csvwriter = csv.writer(csvfile)
    for i in range(len(liftSortedPrint)):
        csvwriter.writerow(liftSortedPrint[i])
        i = i + 1
csvfile.close()

## Max Total Confidence Sort
Sort by overall max confidence

In [223]:
#Sorting rules by max total confidence across all subreddits.
printOCravel = []
for i in range(len(printOrderedColumns)):
    printOCravel.append([y for x in printOrderedColumns[i] for y in x])
printOCravelCopy = printOCravel
del printOCravelCopy[0]
del printOCravelCopy[0]

#Sorting rules by max total confidence across all subreddits.
#Summing Lifts
for row in printOCravelCopy:
    row.append(float(row[2]) + float(row[7]) + float(row[12]) + float(row[17]) + float(row[22]))
    
#Add Labels
labels = []
for i in range(2):
    labels.append([y for x in printOrderedColumns[i] for y in x])
labels[1].append('Confidence Sum')
labels[0].append('')
labels[0].append('')
print(labels)

#Sort by the sum calculated
printOCravelCopy[4]
liftSortedPrint = sorted(printOCravelCopy, key=lambda x: -x[25])
liftSortedPrint = labels + liftSortedPrint

[['Subreddit: Marijuana', '', '', '', ' ', 'Subreddit: Cigarettes', '', '', '', ' ', 'Subreddit: Alcohol', '', '', '', ' ', 'Subreddit: Opiates', '', '', '', ' ', 'Subreddit: Cocaine', '', '', '', ' ', '', ''], ['Rule', 'Support (%)', 'Confidence', 'Lift', ' ', 'Rule', 'Support (%)', 'Confidence', 'Lift', ' ', 'Rule', 'Support (%)', 'Confidence', 'Lift', ' ', 'Rule', 'Support (%)', 'Confidence', 'Lift', ' ', 'Rule', 'Support (%)', 'Confidence', 'Lift', ' ', 'Confidence Sum']]


In [224]:
#Write to a CSV File
import csv
import pandas as pd
with open('Association-Analysis/'+model_name+'Associations'+str(s_threshold)+'S'+str(c_threshold)+'C_Threshold_MaxConfidenceSort.csv','w') as csvfile:
    csvwriter = csv.writer(csvfile)
    for i in range(len(liftSortedPrint)):
        csvwriter.writerow(liftSortedPrint[i])
        i = i + 1
csvfile.close()
#df = pd.DataFrame(printFormat)
#df.to_csv('testFile.csv', index=False)

### Max Confidence by Subreddit Sort
Sort by subreddit max confidence.

In [225]:
#Sorting rules by one subreddit's max confidence.
for subName in subreddit_names:
    subredditChoice = subName
    index = int(subreddit_names.index(subredditChoice))
    printOCravel = []
    for i in range(len(printOrderedColumns)):
        printOCravel.append([y for x in printOrderedColumns[i] for y in x])
    printOCravelCopy = printOCravel
    del printOCravelCopy[0]
    del printOCravelCopy[0]

    #Sorting rules by max confidence of specified subreddit.
    #Add Labels and some space between subreddits
    labels = []
    for i in range(2):
        #this removes nested lists
        labels.append([y for x in printOrderedColumns[i] for y in x])
    labels[1].append('Lifts Sum')
    labels[0].append('')
    labels[0].append('')
    
    #Sort by the sum calculated
    sortIndex = 5*index + 2
    printOCravelCopy[4]
    liftSortedPrint = sorted(printOCravelCopy, key=lambda x: -float(x[sortIndex]))
    liftSortedPrint = labels + liftSortedPrint
    
    #Write to a CSV File
    import csv
    import pandas as pd
    with open('Association-Analysis/'+model_name+'Associations'+str(s_threshold)+'S'+str(c_threshold)+'C_Threshold_MaxConfidence' + subredditChoice +'.csv','w') as csvfile:
        csvwriter = csv.writer(csvfile)
        for i in range(len(liftSortedPrint)):
            csvwriter.writerow(liftSortedPrint[i])
            i = i + 1
    csvfile.close()

### Max Lift by Subreddit Sort
Sort by subreddit max lift

In [232]:
#Sorting rules by one subreddit's max lift.
for subName in subreddit_names:
    subredditChoice = subName
    index = int(subreddit_names.index(subredditChoice))
    printOCravel = []
    for i in range(len(printOrderedColumns)):
        printOCravel.append([y for x in printOrderedColumns[i] for y in x])
    printOCravelCopy = printOCravel
    del printOCravelCopy[0]
    del printOCravelCopy[0]

    #Sorting rules by max confidence of specified subreddit.
    #Add Labels and space between subreddits
    labels = []
    for i in range(2):
        #this removes nested lists
        labels.append([y for x in printOrderedColumns[i] for y in x])
    labels[1].append('Lifts Sum')
    labels[0].append('')
    labels[0].append('')
    
    #Sort by the sum calculated
    sortIndex = 5*index + 3
    printOCravelCopy[4]
    liftSortedPrint = sorted(printOCravelCopy, key=lambda x: -float(x[sortIndex]))
    liftSortedPrint = labels + liftSortedPrint
    
    #Write to a CSV File
    import csv
    import pandas as pd
    with open('Association-Analysis/'+model_name+'Associations'+str(s_threshold)+'S'+str(c_threshold)+'C_Threshold_MaxLift' + subredditChoice +'.csv','w') as csvfile:
        csvwriter = csv.writer(csvfile)
        for i in range(len(liftSortedPrint)):
            csvwriter.writerow(liftSortedPrint[i])
            i = i + 1
    csvfile.close()

## Graphing the Results of Original Rule Clusters
Here we will be graphing the LHS and RHS distributions of rules clusters for the association rules originally generated for the corresponding subreddits. We didn't end up using this code in our paper. May not be necessary but it's here if you need it.

In [368]:
lhs_freq_dicts = [{},{},{},{},{}]
index = 0
for rulebook in rules_slim:
    for rule in rulebook:
        try:
            add = lhs_freq_dicts[index][rule[0]]
            lhs_freq_dicts[index].update({rule[0]:(add + 1)})
        except:
            lhs_freq_dicts[index].update({rule[0]:1})
    index += 1

In [369]:
rhs_freq_dicts = [{},{},{},{},{}]
index = 0
for rulebook in rules_slim:
    for rule in rulebook:
        try:
            add = rhs_freq_dicts[index][rule[1]]
            rhs_freq_dicts[index].update({rule[1]:(add + 1)})
        except:
            rhs_freq_dicts[index].update({rule[1]:1})
    index += 1

In [371]:
#rhs_freq_dicts[0]

In [372]:
sorted_rhs_tuples = []
sorted_lhs_tuples = []
for i in range(len(rhs_freq_dicts)):
    sorted_rhs_tuples.append(sorted(rhs_freq_dicts[i].items(), key=operator.itemgetter(1)))
    sorted_lhs_tuples.append(sorted(lhs_freq_dicts[i].items(), key=operator.itemgetter(1)))

In [382]:
x_clustersLHS = []
y_freqLHS = []
x_clustersRHS = []
y_freqRHS = []

#Right Hand Side Stuff
for sortedtuples in sorted_rhs_tuples:
    x_clustersSubr = []
    y_freqSubr = []
    for i in range(len(sortedtuples)):
        #x_clustersLHS.append(list(sorted_lhs_tuples[0][i][0])[0])
        x_clustersSubr.append(list(sortedtuples[i][0])[0] + 1)
        y_freqSubr.append(sortedtuples[i][1])
    x_clustersRHS.append(x_clustersSubr)
    y_freqRHS.append(y_freqSubr)
    index += 1

#Left Hand Side Stuff
for sortedtuples in sorted_lhs_tuples:
    x_clustersSubr = []
    y_freqSubr = []
    for i in range(len(sortedtuples)):
        x_clustersSubr.append(list(sortedtuples[i][0])[0] + 1)
        y_freqSubr.append(sortedtuples[i][1])
    x_clustersLHS.append(x_clustersSubr)
    y_freqLHS.append(y_freqSubr)
    index += 1

In [383]:
#Normalize Data to Length of Rule set.
i = 0
for lists in y_freqRHS:
    lists[:] = [x*100 / len(rules_slim[i]) for x in lists]
    i += 1
i = 0
for lists in y_freqLHS:
    lists[:] = [x*100 / len(rules_slim[i]) for x in lists]
    i += 1

In [391]:
#Enter colors for your corresponding subreddits
colors = [(0.133, 0.584, 0.882),(0.631, 0.133, 0.882),(0.745, 0.066, 0.035),(0.901, 0.635, 0.101),(0.133, 0.882, 0.607)]

#initializing variables/Adjusting parameters
x = np.linspace(1,100,100)
w = 0.8/5
index = 0
x_Shift = -2.5*w
fig = plt.figure(figsize=(200,100))
ax = plt.subplot(111)
plt.xticks(x)
ax.tick_params(axis='both', which='major', labelsize=50)
ax.set_title('Drug Abuse RHS Rule Cluster Usage',fontsize=100,fontweight='bold',y=1.02)
ax.set_xlabel('Cluster Number',fontsize=100,y=1)
ax.set_ylabel('% Cluster',fontsize=100)
ax.yaxis.set_label_coords(0.112, 0.5, transform=fig.transFigure)
ax.xaxis.set_label_coords(0.513, 0.1, transform=fig.transFigure)

#Graph Results
index = 0
for frequencies in y_freqRHS:
    ax.bar(np.asarray(x_clustersRHS[index])+x_Shift, frequencies, width=w, color=colors[index], align='center',label=subreddit_names[index])
    x_Shift=x_Shift+w
    index +=1

plt.legend(loc ='upper right',prop={'size':100})
plt.show()

In [390]:
#Enter colors for your corresponding subreddits
colors = [(0.133, 0.584, 0.882),(0.631, 0.133, 0.882),(0.745, 0.066, 0.035),(0.901, 0.635, 0.101),(0.133, 0.882, 0.607)]

#initializing variables/Adjusting parameters
x = np.linspace(1,100,100)
w = 0.8/5
index = 0
x_Shift = -2.5*w
fig = plt.figure(figsize=(200,100))
ax = plt.subplot(111)
plt.xticks(x)
ax.tick_params(axis='both', which='major', labelsize=50)
ax.set_title('Drug Abuse LHS Rule Cluster Usage',fontsize=100,fontweight='bold',y=1.02)
ax.set_xlabel('Cluster Number',fontsize=100,y=1)
ax.set_ylabel('% Cluster',fontsize=100)
ax.yaxis.set_label_coords(0.112, 0.5, transform=fig.transFigure)
ax.xaxis.set_label_coords(0.513, 0.1, transform=fig.transFigure)

#Graph Results
index = 0
for frequencies in y_freqLHS:
    ax.bar(np.asarray(x_clustersLHS[index])+x_Shift, frequencies, width=w, color=colors[index], align='center',label=subreddit_names[index])
    x_Shift=x_Shift+w
    index +=1

plt.legend(loc ='upper left',prop={'size':100})
plt.show()

### Extra Stuff
This is checks that lifts and such worked properly. Manual computation of association rules and printing out words from a post.


In [ ]:
#Skip these down until the create association rules heading. These were used to find the text for a specific post,
# and to do apriori on one rule to check that the python package is working properly.
firstPostWords = ''
print(PostsByWords[0][0].shape[1])
for i in range(PostsByWords[0][0].shape[1]):
    if PostsByWords[0][0][0,i] != 0:
        firstPostWords += vocab_list[i] + ' '
print(firstPostWords)    



In [ ]:
Cluster98Words = []
print(PostsByClusters[0][0].todense())
for i in range(ClustersByWords[97].shape[1]):
    if ClustersByWords[97][0,i] != 0:
        Cluster98Words.append(vocab_list[i])

In [ ]:
import string
s = "I'm 18 and I've been smoking everyday for about 2 years now. It used to be just at night, then maybe once in the afternoon, then every morning, and now I smoke before everything I do. Smoking has become so heavily engrained into my daily routine that, when I am in a situation where I can't smoke, it has become a problem. The main thing is my appetite. I feel like the only time I can eat, is after I smoke. Leading me to not be able to eat any other time. If I try to eat and I haven't smoked, I can't even do it, all food just tastes wrong and I have to force myself to swallow. This summer I went on a weeklong vacation with my parents and I couldn't bring any bud. During the day I felt fine (and didn't eat a thing) but at night time I would get very sad, depressed, and anxious. It was a horrible week, and it showed me how much of a problem I really had. What makes it even harder is that my parents are totally okay with me smoking. They weren't at first, but now it's gotten to the point where my dad has bought me bongs and regularly gives me money for weed. Plus, all my friends are regular smokers too, just not as much as me. On top of it all, as much as I don't want to say it, it definitely effects my grades, but only because I live in dorms this year and I had constant stress on how I was going to keep up my smoking habit, leading to more smoking and less studying. I want to quit, or at least lighten up, only because of this strong dependence I have, but I honestly don't know how. I want to be able to go places, without thinking about how I'm going to bring weed with me to soothe my habit."
s = s.translate(str.maketrans('', '', string.punctuation))
s = s.split()
keyWord = Cluster98Words
outString = ''
for word in s:
    if word in keyWord:
        print(word)



In [ ]:
#Get Support values for specific rules and update the itemsets dictionaries.
def get_support_2_cluster_WithFavorance(array,PostsByClusters,itemsets):
    index1 = 0
    for itemset in itemsets:
        supportcount = 0
        try:
            itemset[frozenset({(array[0] - 1),(array[1] - 1)})]
            index1 = index1 + 1
            continue
        except KeyError:
            for post in PostsByClusters[index1]:
                if post[(0,(array[0] - 1))] != 0 and post[(0,(array[1]-1))] != 0:
                    #product = ([post[(0,array[0])] * post[(0,array[1])]])
                    supportcount += 1
            itemset[frozenset({(array[0] - 1),(array[1] - 1)})] = supportcount
            supportcount = 0
            index1 = index1 + 1
            
def get_support_1_cluster(array,PostsByClusters,itemsets):
    index1 = 0
    for itemset in itemsets:
        supportcount = 0
        try:
            itemset[frozenset({(array[0] - 1)})]
            index1 = index1 + 1
            continue
        except KeyError:
            for post in PostsByClusters[index1]:
                if post[(0,(array[0] - 1))] != 0:
                        supportcount += 1
            itemset[frozenset({(array[0] - 1)})] = supportcount
            supportcount = 0
            index1 = index1 + 1
    return itemsets

In [ ]:
PostsByClustersTop20 = load_object('objects/','DrugAbuse-PostsByClustersTopWords')

In [ ]:
PCMatchedByClusters = load_object('matricies/','2019_SW_PandC-PCMatchedByClusters-100clusters')
import sparse
import scipy
PCMatched = scipy.sparse.csr_matrix(PCMatchedByClusters)
PCMatchedByClusters[0]

In [ ]:
PCByClusters = PCMatchedByClusters

In [ ]:
itemsDict = [{},{},{},{},{}]
testClusters = [2,97]
get_support_2_cluster_WithFavorance(testClusters,PostsByClusters,itemsDict)
get_support_1_cluster([testClusters[0]],PostsByClusters,itemsDict)
get_support_1_cluster([testClusters[1]],PostsByClusters,itemsDict)

In [ ]:
print(itemsDict)

In [ ]:
lifts = []
support = []
for i in range(5):
    support.append(itemsDict[i][frozenset({testClusters[0] - 1, testClusters[1] - 1})]/ PostsByClusters[i].shape[0])
    lifts.append(itemsDict[i][frozenset({testClusters[0] - 1, testClusters[1] - 1})] / (itemsDict[i][frozenset({testClusters[1] - 1})] * itemsDict[i][frozenset({testClusters[0] - 1})]) * PostsByClusters[i].shape[0])

In [ ]:
print(lifts)
print(support)